In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
from timeit import timeit
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates






In [ ]:
###
###
### EXTRACT SHOWER FROM IMAGE
###
###
###


#
#
#
#
#
# Main algorihtm to process events with mask to extract ONLY muon ring
#
#
#
#
#

deleted_pixels = []

#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []


# only fitted events

fitted_contaminated_runs = [8,8,9,9,27,28]
fitted_contaminated_events = [234, 448, 404, 433, 374, 328]
fitted_tel_numbers = [1,1,1,3,2,1]

calib_config = Config({"GlobalPeakWindowSum": {"window_width" :7, "window_shift" : 3}})



for i,j in enumerate(contaminated_runs):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)

    counter = 0
    tel_number = tel_numbers[i]
    cam_geometry = source.subarray.tel[tel_number].camera.geometry

    
    for k,l in enumerate(event_iterator):
        if k == contaminated_events[i]:
            event = l
            events.append(event)
            break
   

    # Building appropriate image for this event
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    #title= f"True Image of Event_{str(contaminated_runs[i])+'run'+str(contaminated_events[i])}"
    title= f"Before returning islands. Event_{str(contaminated_runs[i])+'run'+str(contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    #disp.image = event.simulation.tel[tel_number].true_image
    
    # Зануляємо пікселі з шавером
    pixel_mask = []
    time_bin_separation = 17
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(time_bin_separation,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
            
    working_wave_r1 = event.r1.tel[tel_number].waveform.copy()
    working_wave_r0_0 = event.r0.tel[tel_number].waveform[0].copy()
    working_wave_r0_1 = event.r0.tel[tel_number].waveform[1].copy()
    original_r1_waveform = event.r1.tel[tel_numbers[i]].waveform.copy() # waveform to get back values for islands
    
    
    
    deleted_pixels.append(counter)
    print(len(event.r1.tel[tel_numbers[i]].waveform.sum(axis=1)) - deleted_pixels[0])
    
    for k,l in enumerate(working_wave_r1):
        if pixel_mask[k]:
            working_wave_r1[k] = np.zeros(40)  
            working_wave_r0_0[k] = np.zeros(40)
            working_wave_r0_1[k] = np.zeros(40)
            working_wave_r1[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
    working_waves.append(working_wave_r1)
    
    # записуємо в r1 i r0 контейнер waveform with denoted as shower pixels 
    
    event.r1.tel[tel_numbers[i]].waveform = working_wave_r1
    event.r0.tel[tel_numbers[i]].waveform[0] = working_wave_r0_0
    event.r0.tel[tel_numbers[i]].waveform[1] = working_wave_r0_1
    
    disp.image = event.r1.tel[tel_number].waveform.sum(axis=1)
    #disp.image = event.simulation.tel[tel_number].true_image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)

    
    
    #print("non_zero", np.count_nonzero(working_wave_r1.sum(axis=1)))
    #print("11111event.r1.tel[tel_numbers[i]].waveform = ", event.r1.tel[tel_numbers[i]].waveform.shape)
    
    #print("22222event.r1.tel[tel_numbers[i]].waveform = ", event.r1.tel[tel_numbers[i]].waveform.shape)
    #print("IS NAN", event.dl1.tel[tel_numbers[i]].image)
    
   

    calib(event)
    image_processor(event)


    
    
    # islands cleaning
    #clean_mask = event.dl1.tel[tel_number].image == 0
    clean_mask = event.r1.tel[tel_numbers[i]].waveform.sum(axis=1) == 0
    #print("clean_mask", np.count_nonzero(clean_mask))
    #print("where = ", np.where(clean_mask == True))
#     return_pixels = np.array([])
    
#     n_islands, island_id = number_of_islands(cam_geometry, clean_mask)
    
#     for t in range(1,n_islands+1):
#         if len(np.where(island_id == t)[0]) <= 3:
#             #print(np.where(island_id ==i))
#             return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])
    
#     return_pixels = return_pixels.astype(int)

    
    
#     event.r1.tel[tel_number].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]
    #event.r1.tel[tel_number].waveform[return_pixels,:] = 100
    #event.r1.tel[tel_number].waveform[~return_pixels,:] = 0
    #print("returning pixels len = ", len(original_r1_waveform[return_pixels,:]))
    
    
    #event.r1.tel[tel_number].waveform[return_pixels,:] = 1000
    #print("SDHAPE SHAPE", event.r1.tel[tel_number].waveform[return_pixels,:].shape)
    #
    
    calib(event)
    image_processor(event)
    muon_processor(event)
    
    #opt_effi = []
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
    #opt_effi.append(event.muon.tel[tel_numbers[i]])
    #
    #
    bx = axes[1]
    title= f"Extracted by mask < {time_bin_separation} timebin, optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    #title= f"After returning small islands from the shower mask"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', allow_pick = True, ax=bx)
    #disp.image = working_wave.sum(axis=1)
    disp.image = event.r1.tel[tel_number].waveform.sum(axis = 1)
    #disp.image = event.dl1.tel[tel_number].image
    #disp.image = island_id
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #disp.set_limits_minmax(0,100)
    plt.show()
    #plt.savefig(f"/Users/vdk/ImagePurgatorium/AfterIslandClean_timebin{time_bin_separation}_zero_IslandlessEq3pixels_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    #plt.savefig(f'/Users/vdk/IslandCleanComparing_size<=3_event_{i}')
plt.close()


#print("SHOWER PIXELS = ", counter)
#print("RING PIXELS = ", len(event.r1.tel[tel_numbers[i]].waveform)-counter)

In [ ]:
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)


In [ ]:
# didn't understood why but now:
# Because in first case I count non-zero and in second case I count zeroes
#
test_sum = working_wave_r1.sum(axis=1)
clean_mask_test = working_wave_r1.sum(axis=1) == 0

print(np.count_nonzero(clean_mask_test))
print(np.count_nonzero(test_sum))

for i,j in enumerate(working_wave_r1.sum(axis=1)):
    if j == 0.0 and not clean_mask_test[i]:
        print("here")
        
    #print("working wave = ", j, "  clean_mask  = ", clean_mask_test[i])
    
print(np.count_nonzero(working_wave_r1.sum(axis=1)))
print(np.count_nonzero(clean_mask_test))
print(len(clean_mask_test))
print(len(working_wave_r1))

In [ ]:
for i,j in enumerate(contaminated_runs[:1]):
    counter = 0
    tel_number = tel_numbers[i]
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for k,l in enumerate(event_iterator):
        if k == contaminated_events[i]:
            event = l
            events.append(event)
            break
    # Building appropriate image for this event
    bx = axes[1]
    pixel_mask = []
    time_bin_separation = 17
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(time_bin_separation,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
    working_wave = event.r1.tel[tel_number].waveform.copy()
    for k,l in enumerate(working_wave):
        if pixel_mask[k]:
            working_wave[k] = np.zeros(40)     
            #working_wave[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
    
    event.r1.tel[tel_numbers[i]].waveform = working_wave
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)
    calib(event)
    image_processor(event)
    muon_processor(event)

In [ ]:
#clean_mask = [True for i in event.dl1.tel[tel_number].image if i == 0 else False]
#clean_mask = np.array(event.dl1.tel[tel_number].image, dtype= bool)
len(event.dl1.tel[tel_number].image) - np.count_nonzero(event.dl1.tel[tel_number].image)

In [ ]:
clean_mask = event.dl1.tel[tel_number].image == 0
cam_geometry = source.subarray.tel[tel_number].camera.geometry
print(cam_geometry)

In [ ]:
np.count_nonzero(clean_mask)

In [ ]:
n_islands, island_id = number_of_islands(cam_geometry, clean_mask)
for i in range(1,n_islands+1):
    print(len(np.where(island_id == i)[0]))

In [ ]:
plt.figure()
camgeom = source.subarray.tel[1].camera.geometry
title= f"Extracted by mask <{time_bin_separation}timebin, fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
disp.image = working_wave.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
plt.show()

In [ ]:
picture_islands = clean_mask[]

In [ ]:
## Number of islands with <3 pixels, 3 <= n_pixels <= 50, n_pixels > 50
number_of_island_sizes(island_id)

In [ ]:
from ctapipe.io import EventSource
from ctapipe.utils.datasets import get_dataset_path

input_url = get_dataset_path("gamma_prod5.simtel.zst")

# EventSource() automatically detects what kind of file we are giving it,
# if already supported by ctapipe
source = EventSource(input_url, max_events=5)

print(type(source))

In [ ]:
for event in source:
    print(
        "Id: {}, E = {:1.3f}, Telescopes: {}".format(
            event.count, event.simulation.shower.energy, len(event.r0.tel)
        )
    )

In [ ]:
from ctapipe.calib import CameraCalibrator

calibrator = CameraCalibrator(subarray=source.subarray)

In [ ]:
calibrator(event)

In [ ]:
tel_id=130

In [ ]:
geometry = source.subarray.tel[tel_id].camera.geometry
dl1 = event.dl1.tel[tel_id]

geometry, dl1

In [ ]:
arr1 = np.array([1,2,3,4,5])
arr2 = np.array([1,1,1,1,1])

arr1[[0,1,3]] = arr2[[0,1,3]]

arr1

In [ ]:
from ctapipe.visualization import CameraDisplay

display = CameraDisplay(geometry)

# right now, there might be one image per gain channel.
# This will change as soon as
display.image = dl1.image
display.add_colorbar()

In [ ]:
from ctapipe.image.cleaning import tailcuts_clean

In [ ]:
# unoptimized cleaning levels
cleaning_level = {
    "CHEC": (2, 4, 2),
    "LSTCam": (3.5, 7, 2),
    "FlashCam": (3.5, 7, 2),
    "NectarCam": (4, 8, 2),
}

In [ ]:
boundary, picture, min_neighbors = cleaning_level[geometry.name]

clean = tailcuts_clean(
    geometry,
    dl1.image,
    boundary_thresh=boundary,
    picture_thresh=picture,
    min_number_picture_neighbors=min_neighbors,
)

In [ ]:
clean

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

d1 = CameraDisplay(geometry, ax=ax1)
d2 = CameraDisplay(geometry, ax=ax2)

ax1.set_title("Image")
d1.image = dl1.image
d1.add_colorbar(ax=ax1)

ax2.set_title("Pulse Time")
d2.image = dl1.peak_time - np.average(dl1.peak_time, weights=dl1.image)
d2.cmap = "RdBu_r"
d2.add_colorbar(ax=ax2)
d2.set_limits_minmax(-20, 20)

d1.highlight_pixels(clean, color="red", linewidth=1)

In [ ]:
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates

In [ ]:
hillas = hillas_parameters(geometry[clean], dl1.image[clean])

print(hillas)

In [ ]:
len(dl1.image[clean])

In [ ]:
display = CameraDisplay(geometry)

# set "unclean" pixels to 0
cleaned = dl1.image.copy()
cleaned[~clean] = 0.0

display.image = cleaned
display.add_colorbar()

display.overlay_moments(hillas, color="xkcd:red")

In [ ]:
timing = timing_parameters(geometry, dl1.image, dl1.peak_time, hillas, clean)

print(timing)

In [ ]:
long, trans = camera_to_shower_coordinates(
    geometry.pix_x, geometry.pix_y, hillas.x, hillas.y, hillas.psi
)

plt.plot(long[clean], dl1.peak_time[clean], "o")
plt.plot(long[clean], timing.slope * long[clean] + timing.intercept)

In [ ]:
l = leakage_parameters(geometry, dl1.image, clean)
print(l)

In [ ]:
disp = CameraDisplay(geometry)
disp.image = dl1.image
disp.highlight_pixels(geometry.get_border_pixel_mask(1), linewidth=2, color="xkcd:red")

In [ ]:
n_islands, island_id = number_of_islands(geometry, clean)

print(n_islands)

In [ ]:
np.where(island_id == 2)

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord, AltAz

from ctapipe.containers import ImageParametersContainer
from ctapipe.io import EventSource
from ctapipe.utils.datasets import get_dataset_path

from ctapipe.calib import CameraCalibrator

from ctapipe.image import ImageProcessor
from ctapipe.reco import ShowerProcessor

from ctapipe.io import DataWriter

from copy import deepcopy
import tempfile

from traitlets.config import Config


image_processor_config = Config(
    {
        "ImageProcessor": {
            "image_cleaner_type": "TailcutsImageCleaner",
            "TailcutsImageCleaner": {
                "picture_threshold_pe": [
                    ("type", "LST_LST_LSTCam", 7.5),
                    ("type", "MST_MST_FlashCam", 8),
                    ("type", "MST_MST_NectarCam", 8),
                    ("type", "SST_ASTRI_CHEC", 7),
                ],
                "boundary_threshold_pe": [
                    ("type", "LST_LST_LSTCam", 5),
                    ("type", "MST_MST_FlashCam", 4),
                    ("type", "MST_MST_NectarCam", 4),
                    ("type", "SST_ASTRI_CHEC", 4),
                ],
            },
        }
    }
)

input_url = get_dataset_path("gamma_prod5.simtel.zst")
source = EventSource(input_url)

calibrator = CameraCalibrator(subarray=source.subarray)
image_processor = ImageProcessor(
    subarray=source.subarray, config=image_processor_config
)
shower_processor = ShowerProcessor(subarray=source.subarray)
horizon_frame = AltAz()

f = tempfile.NamedTemporaryFile(suffix=".hdf5")

with DataWriter(
    source, output_path=f.name, overwrite=True, write_showers=True
) as writer:

    for event in source:
        energy = event.simulation.shower.energy
        n_telescopes_r1 = len(event.r1.tel)
        event_id = event.index.event_id
        print(f"Id: {event_id}, E = {energy:1.3f}, Telescopes (R1): {n_telescopes_r1}")

        calibrator(event)
        image_processor(event)
        shower_processor(event)

        stereo = event.dl2.stereo.geometry["HillasReconstructor"]
        if stereo.is_valid:
            print("  Alt: {:.2f}°".format(stereo.alt.deg))
            print("  Az: {:.2f}°".format(stereo.az.deg))
            print("  Hmax: {:.0f}".format(stereo.h_max))
            print("  CoreX: {:.1f}".format(stereo.core_x))
            print("  CoreY: {:.1f}".format(stereo.core_y))
            print("  Multiplicity: {:d}".format(len(stereo.telescopes)))

        # save a nice event for plotting later
        if event.count == 3:
            plotting_event = deepcopy(event)

        writer(event)

In [ ]:
from astropy.coordinates.angle_utilities import angular_separation
import pandas as pd

from ctapipe.io import TableLoader

loader = TableLoader(f.name, load_dl2=True, load_simulated=True)

events = loader.read_subarray_events()

In [ ]:
theta = angular_separation(
    events["HillasReconstructor_az"].quantity,
    events["HillasReconstructor_alt"].quantity,
    events["true_az"].quantity,
    events["true_alt"].quantity,
)

plt.hist(theta.to_value(u.deg) ** 2, bins=25, range=[0, 0.3])
plt.xlabel(r"$\theta² / deg²$")
None

In [ ]:
from ctapipe.visualization import ArrayDisplay


angle_offset = plotting_event.pointing.array_azimuth

plotting_hillas = {
    tel_id: dl1.parameters.hillas for tel_id, dl1 in plotting_event.dl1.tel.items()
}

plotting_core = {
    tel_id: dl1.parameters.core.psi for tel_id, dl1 in plotting_event.dl1.tel.items()
}


disp = ArrayDisplay(source.subarray)

disp.set_line_hillas(plotting_hillas, plotting_core, 500)

plt.scatter(
    plotting_event.simulation.shower.core_x,
    plotting_event.simulation.shower.core_y,
    s=200,
    c="k",
    marker="x",
    label="True Impact",
)
plt.scatter(
    plotting_event.dl2.stereo.geometry["HillasReconstructor"].core_x,
    plotting_event.dl2.stereo.geometry["HillasReconstructor"].core_y,
    s=200,
    c="r",
    marker="x",
    label="Estimated Impact",
)

plt.legend()
# plt.xlim(-400, 400)
# plt.ylim(-400, 400)

In [ ]:
from ctapipe.image import toymodel
from ctapipe.instrument import SubarrayDescription


geometry = loader.subarray.tel[1].camera.geometry

In [ ]:
N = 20000
7.5/(N**0.5)

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

In [ ]:
source.subarray.peek()

In [ ]:
150*600

In [ ]:
600*600

In [ ]:
400*400

In [ ]:
100*100

In [ ]:
10000*40

In [ ]:
15*60

In [ ]:
400000/900

In [ ]:
444*470

In [ ]:
470*40


In [ ]:
test1 = np.array([1,2,3,4,5])
test2 = np.array([0,0,0,0,0])

indices = np.array([0,1,2])


#np.put(test1,indices,test2)
print(test1)

test1[indices] = test2[indices]
print(test1)

In [ ]:
return_pixels

In [ ]:
test1[indices] = 0

In [ ]:
event.r1.tel[tel_number].waveform[[1,2,3],:]

In [ ]:
###
###
### EXTRACT SHOWER FROM IMAGE
###
###
###


#
#
#
#
#
# WORKING ISLAND CHOOSER
#
#
#
#
#

deleted_pixels = []

#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []


# only fitted events

fitted_contaminated_runs = [8,8,9,9,27,28]
fitted_contaminated_events = [234, 448, 404, 433, 374, 328]
fitted_tel_numbers = [1,1,1,3,2,1]

calib_config = Config({"GlobalPeakWindowSum": {"window_width" :7, "window_shift" : 3}})



for i,j in enumerate(fitted_contaminated_runs):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    tel_number = fitted_tel_numbers[i]
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)
    cam_geometry = source.subarray.tel[tel_number].camera.geometry
    counter = 0

    
    
    
    for k,l in enumerate(event_iterator):
        if k == fitted_contaminated_events[i]:
            event = l
            events.append(event)
            break
   

    # Building appropriate image for this event
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"True Image of Event_{str(fitted_contaminated_runs[i])+'run'+str(fitted_contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    #disp.image = event.simulation.tel[tel_number].true_image
    
    # Зануляємо пікселі з шавером
    pixel_mask = []
    time_bin_separation = 17
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(time_bin_separation,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
            
    working_wave_r1 = event.r1.tel[tel_number].waveform.copy()
    working_wave_r0_0 = event.r0.tel[tel_number].waveform[0].copy()
    working_wave_r0_1 = event.r0.tel[tel_number].waveform[1].copy()
    original_r1_waveform = event.r1.tel[tel_number].waveform.copy() # waveform to get back values for islands
    
    
    
    deleted_pixels.append(counter)
    print(len(event.r1.tel[fitted_tel_numbers[i]].waveform.sum(axis=1)) - deleted_pixels[0])
    
    for k,l in enumerate(working_wave_r1):
        if pixel_mask[k]:
            #working_wave_r1[k] = np.zeros(40)  
            working_wave_r0_0[k] = np.zeros(40)
            working_wave_r0_1[k] = np.zeros(40)
            working_wave_r1[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
    working_waves.append(working_wave_r1)
    
    # записуємо в r1 i r0 контейнер waveform with denoted as shower pixels 
    
    event.r1.tel[tel_number].waveform = working_wave_r1
    event.r0.tel[tel_number].waveform[0] = working_wave_r0_0
    event.r0.tel[tel_number].waveform[1] = working_wave_r0_1
    
    disp.image = event.r1.tel[tel_number].waveform.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)

    
    
    #print("non_zero", np.count_nonzero(working_wave_r1.sum(axis=1)))
    #print("11111event.r1.tel[tel_numbers[i]].waveform = ", event.r1.tel[tel_numbers[i]].waveform.shape)
    
    #print("22222event.r1.tel[tel_numbers[i]].waveform = ", event.r1.tel[tel_numbers[i]].waveform.shape)
    #print("IS NAN", event.dl1.tel[tel_numbers[i]].image)
    
    #print("Before calib", event.dl1.tel[tel_numbers[i]].image)
    calib(event)
    #print("After calib", event.dl1.tel[tel_numbers[i]].image)
    image_processor(event)
    
    #
    # islands cleaning
    #
    #clean_mask = event.dl1.tel[tel_number].image == 0
    
    
    clean_mask = event.r1.tel[tel_number].waveform.sum(axis=1) == 0
    #clean_mask = event.r1.tel[tel_number].waveform.sum(axis=1) == np.mean(event.r1.tel[tel_number].waveform, axis = 0).sum()
    print("Clean mask len = ", len(clean_mask))
    
    #print("shape 1 ", event.r1.tel[tel_number].waveform.sum(axis=1).shape)
    #print("shape 2 ", np.mean(event.r1.tel[tel_number].waveform, axis = 0).sum().shape)
    #print("clean_mask", np.count_nonzero(clean_mask))
    #print("where = ", np.where(clean_mask == True))
    return_pixels = np.array([])
    
    n_islands, island_id = number_of_islands(cam_geometry, clean_mask)
    
    for t in range(1,n_islands+1):
        if len(np.where(island_id == t)[0]) <= 3:
            #print(np.where(island_id ==i))
            return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])
    
    return_pixels = return_pixels.astype(int)
    #print("len return", len(return_pixels))
    #print(return_pixels.astype(int))
    
    
    event.r1.tel[tel_number].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]
    #event.r1.tel[tel_number].waveform[return_pixels,:] = 1
    #event.r1.tel[tel_number].waveform[~return_pixels,:] = 0
    #print("returning pixels len = ", len(original_r1_waveform[return_pixels,:]))
    
    
    #event.r1.tel[tel_number].waveform[return_pixels,:] = 1000
    #print("SDHAPE SHAPE", event.r1.tel[tel_number].waveform[return_pixels,:].shape)
    #
    
    calib(event)
    image_processor(event)
    muon_processor(event)
    
    #opt_effi = []
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
    #opt_effi.append(event.muon.tel[tel_numbers[i]])
    #
    #
    bx = axes[1]
    title= f"Extracted by mask <{time_bin_separation}timebin, fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
    #disp.image = working_wave.sum(axis=1)
    disp.image = event.r1.tel[tel_number].waveform.sum(axis = 1)
    #disp.image = event.dl1.tel[tel_number].image
    #disp.image = island_id
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()
    #plt.savefig(f"/Users/vdk/ImagePurgatorium/AfterIslandClean_timebin{time_bin_separation}_zero_IslandlessEq3pixels_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    
plt.close()


#print("SHOWER PIXELS = ", counter)
#print("RING PIXELS = ", len(event.r1.tel[tel_numbers[i]].waveform)-counter)

In [ ]:
bin17_cleaned_zero = [0.163, np.nan, np.nan, 0.179, 0.185, 0.192]
raw = [0.165, 0.178, 0.194, 0.178, 0.188, 0.195]
bin17_zero = [np.nan, np.nan, 0.188, np.nan, 0.185, np.nan]

In [ ]:
###
###
### EXTRACT SHOWER FROM IMAGE
###
###
###


#
#
#
#
#
# Main algorihtm to process events with mask to extract ONLY muon ring
#
#
#
#
#

deleted_pixels = []

#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []


# only fitted events

fitted_contaminated_runs = [8,8,9,9,27,28]
fitted_contaminated_events = [234, 448, 404, 433, 374, 328]
fitted_tel_numbers = [1,1,1,3,2,1]

calib_config = Config({"GlobalPeakWindowSum": {"window_width" :7, "window_shift" : 3}})



for i,j in enumerate(fitted_contaminated_runs):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    tel_number = fitted_tel_numbers[i]
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)
    cam_geometry = source.subarray.tel[tel_number].camera.geometry
    counter = 0

    
    
    
    for k,l in enumerate(event_iterator):
        if k == fitted_contaminated_events[i]:
            event = l
            events.append(event)
            break
   

    # Building appropriate image for this event
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"True Image of Event_{str(fitted_contaminated_runs[i])+'run'+str(fitted_contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    #disp.image = event.simulation.tel[tel_number].true_image
    
    # Зануляємо пікселі з шавером
    pixel_mask = []
    time_bin_separation = 17
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(time_bin_separation,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
            
    working_wave_r1 = event.r1.tel[tel_number].waveform.copy()
    working_wave_r0_0 = event.r0.tel[tel_number].waveform[0].copy()
    working_wave_r0_1 = event.r0.tel[tel_number].waveform[1].copy()
    original_r1_waveform = event.r1.tel[tel_number].waveform.copy() # waveform to get back values for islands
    
    
    
    deleted_pixels.append(counter)
    print(len(event.r1.tel[fitted_tel_numbers[i]].waveform.sum(axis=1)) - deleted_pixels[0])
    for_clean_mask = np.array([]).astype(bool)
    
    for k,l in enumerate(working_wave_r1):
        if pixel_mask[k]:
            for_clean_mask = np.append(for_clean_mask, True)
            working_wave_r1[k] = np.zeros(40)  
            working_wave_r0_0[k] = np.zeros(40)
            working_wave_r0_1[k] = np.zeros(40)
            
            #working_wave_r0_0[k] = np.mean(event.r0.tel[tel_number].waveform[0], axis = 0)
            #working_wave_r0_1[k] = np.mean(event.r0.tel[tel_number].waveform[1], axis = 0)
            #working_wave_r1[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
        else:
            for_clean_mask = np.append(for_clean_mask, False)
    working_waves.append(working_wave_r1)
    
    print("LEN FOR CLEAN MASK ", len(for_clean_mask))
    # записуємо в r1 i r0 контейнер waveform with denoted as shower pixels 
    
    event.r1.tel[tel_number].waveform = working_wave_r1
    event.r0.tel[tel_number].waveform[0] = working_wave_r0_0
    event.r0.tel[tel_number].waveform[1] = working_wave_r0_1
    
    disp.image = event.r1.tel[tel_number].waveform.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)

    
    
    #print("non_zero", np.count_nonzero(working_wave_r1.sum(axis=1)))
    #print("11111event.r1.tel[tel_numbers[i]].waveform = ", event.r1.tel[tel_numbers[i]].waveform.shape)
    
    #print("22222event.r1.tel[tel_numbers[i]].waveform = ", event.r1.tel[tel_numbers[i]].waveform.shape)
    #print("IS NAN", event.dl1.tel[tel_numbers[i]].image)
    
    #print("Before calib", event.dl1.tel[tel_numbers[i]].image)
    calib(event)
    #print("After calib", event.dl1.tel[tel_numbers[i]].image)
    image_processor(event)
    
    #
    # islands cleaning
    #
    #clean_mask = event.dl1.tel[tel_number].image == 0
    
    
    clean_mask = event.r1.tel[tel_number].waveform.sum(axis=1) == 0
    #clean_mask = for_clean_mask
    #clean_mask = event.r1.tel[tel_number].waveform.sum(axis=1) == np.mean(event.r1.tel[tel_number].waveform, axis = 0).sum()
    print("Clean mask len = ", len(clean_mask))
    print("len cam geometry = ", len(cam_geometry))
    
    #print("shape 1 ", event.r1.tel[tel_number].waveform.sum(axis=1).shape)
    #print("shape 2 ", np.mean(event.r1.tel[tel_number].waveform, axis = 0).sum().shape)
    #print("clean_mask", np.count_nonzero(clean_mask))
    #print("where = ", np.where(clean_mask == True))
    return_pixels = np.array([])
    
    n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)
    
    for t in range(1,n_islands+1):
        if len(np.where(island_id == t)[0]) <= 3:
            #print(np.where(island_id ==i))
            return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])
    
    return_pixels = return_pixels.astype(int)
    #print("len return", len(return_pixels))
    #print(return_pixels.astype(int))
    

    event.r1.tel[tel_number].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]
    
    
    #event.r1.tel[tel_number].waveform[return_pixels,:] = 1
    #event.r1.tel[tel_number].waveform[~return_pixels,:] = 0
    #print("returning pixels len = ", len(original_r1_waveform[return_pixels,:]))
    
    
    #event.r1.tel[tel_number].waveform[return_pixels,:] = 1000
    #print("SDHAPE SHAPE", event.r1.tel[tel_number].waveform[return_pixels,:].shape)
    #
    
    calib(event)
    image_processor(event)
    muon_processor(event)
    
    #opt_effi = []
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
    #opt_effi.append(event.muon.tel[tel_numbers[i]])
    #
    #
    bx = axes[1]
    title= f"Extracted by mask <{time_bin_separation}timebin, fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
    #disp.image = working_wave.sum(axis=1)
    disp.image = event.r1.tel[tel_number].waveform.sum(axis = 1)
    #disp.image = event.dl1.tel[tel_number].image
    #disp.image = island_id
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()
    #plt.savefig(f"/Users/vdk/ImagePurgatorium/AfterIslandClean_timebin{time_bin_separation}_zero_IslandlessEq3pixels_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    
plt.close()


#print("SHOWER PIXELS = ", counter)
#print("RING PIXELS = ", len(event.r1.tel[tel_numbers[i]].waveform)-counter)

In [ ]:
###
###
### FINAL VERSION OF CLEANER FOR TESTS TESTS TESTS
###
###
###


#
#
#
#
#
# Main algorihtm to process events with mask to extract ONLY muon ring
#
#
#
#
#

deleted_pixels = []
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []

# only fitted events

fitted_contaminated_runs = [8,8,9,9,27,28]
fitted_contaminated_events = [234, 448, 404, 433, 374, 328]
fitted_tel_numbers = [1,1,1,3,2,1]

calib_config = Config({"GlobalPeakWindowSum": {"window_width" :7, "window_shift" : 3}})



for i,j in enumerate(fitted_contaminated_runs):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    tel_number = fitted_tel_numbers[i]
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)
    cam_geometry = source.subarray.tel[tel_number].camera.geometry
    counter = 0

    
    
    # Extract required event from dataset
    for k,l in enumerate(event_iterator):
        if k == fitted_contaminated_events[i]:
            event = l
            events.append(event)
            break
   
        
    # Building appropriate image for this event
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"True Image of Event_{str(fitted_contaminated_runs[i])+'run'+str(fitted_contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    # Plot the original/initial image
    
    
    
    
    # Зануляємо пікселі з шавером
    pixel_mask = [] # mask for storing shower pixels as True values
    time_bin_separation = 17
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(time_bin_separation,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
    
    
    
    #NSB_mask = np.array([True for i in range(0,1856)])
    NSB_mask = []
    for k,l in enumerate(event.r1.tel[tel_number].waveform):
        if np.argmax(l) in range(32,40): 
            NSB_mask.append(l)
    NSB_mask = np.array(NSB_mask)
    print(NSB_mask.shape)
    
    
    
    # Create waveform templates to have initial values stored somewhere    
    working_wave_r1 = event.r1.tel[tel_number].waveform.copy()
    working_wave_r0_0 = event.r0.tel[tel_number].waveform[0].copy()
    working_wave_r0_1 = event.r0.tel[tel_number].waveform[1].copy()
    original_r1_waveform = event.r1.tel[tel_number].waveform.copy() # waveform to get back values for islands
    
    
     
    deleted_pixels.append(counter) # array to store how much pixels we delete from the event
    for_clean_mask = np.array([]).astype(bool) # ? Перевірити чи нам взагалі потрібна ця
    NSB_mask = np.array(NSB_mask)
    print("SHAPE ", NSB_mask.shape)
    # In this cycle we change values in the pixels with shower to some other values
    for k,l in enumerate(working_wave_r1):
        if pixel_mask[k]:
            for_clean_mask = np.append(for_clean_mask, True)
            #working_wave_r1[k] = np.zeros(40)  
            #working_wave_r0_0[k] = np.zeros(40)
            #working_wave_r0_1[k] = np.zeros(40)
            
            
            working_wave_r1[k] = np.mean(NSB_mask, axis = 0)
            #working_wave_r1[k] = np.zeros(40)
            working_wave_r0_0[k] = np.zeros(40)
            working_wave_r0_1[k] = np.zeros(40)
            #print("Mean NSB = ", np.mean(NSB_mask, axis = 0))
            # HERE NEED TO IMPROVE - ASSIGN PROPER VALUES TO THE SHOWER PIXELS
            
            #working_wave_r0_0[k] = np.mean(event.r0.tel[tel_number].waveform[0], axis = 0)
            #working_wave_r0_1[k] = np.mean(event.r0.tel[tel_number].waveform[1], axis = 0)
            #working_wave_r1[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
        else:
            for_clean_mask = np.append(for_clean_mask, False)
    working_waves.append(working_wave_r1)
    
    print("LEN FOR CLEAN MASK ", len(for_clean_mask))

    # записуємо в r1 i r0 контейнер waveform with changed shower pixels 
    event.r1.tel[tel_number].waveform = working_wave_r1
    event.r0.tel[tel_number].waveform[0] = working_wave_r0_0
    event.r0.tel[tel_number].waveform[1] = working_wave_r0_1
    


    
    # Calibrate event from R1 to DL1 data
    calib(event)
    
    #MAYBE WE NEED HERE JUST A TAILCUT CLEANER AS IN THE TUTORIAL?
    
    
    #
    # islands cleaning
    #
    #
    
    # Array for storing pixels for return
    return_pixels = np.array([])
    
    n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)
    
    # If size of the islands less then some value, then we append pixels of these islands to the array return_pixels
    for t in range(1,n_islands+1):
        if len(np.where(island_id == t)[0]) <= 3:
            return_pixels = np.append(return_pixels,np.where(island_id == t)[0])
    
    # Change the type to the int
    return_pixels = return_pixels.astype(int)
    
    #
    #
    # Assign original values to the islands which has sizes less then some value
    event.r1.tel[tel_number].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]
    #
    #
    #
    
    
    # Whole optical throughput pipeline (R1 - DL1 -> Cleaning/Parametrizing -> MuonFitting)
    calib(event)
    image_processor(event)
    muon_processor(event)

    #disp.image = event.simulation.tel[tel_number].true_image
    disp.image = event.dl1.tel[tel_number].image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)

    bx = axes[1]
    title= f"Extracted by mask <{time_bin_separation}timebin, fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
    #disp.image = event.r1.tel[tel_number].waveform.sum(axis = 1)
    disp.image = event.r1.tel[tel_number].waveform.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()
    #plt.savefig(f"/Users/vdk/ImagePurgatorium/AfterIslandClean_timebin{time_bin_separation}_zero_IslandlessEq3pixels_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    #plt.savefig(f"/Users/vdk/ImagePurgatorium/CheckWithOUTImageProcessor_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    
plt.close()


#print("SHOWER PIXELS = ", counter)
#print("RING PIXELS = ", len(event.r1.tel[tel_numbers[i]].waveform)-counter)

In [ ]:
clean_mask